In [1]:
import math
import copy
import torch
from torch import nn, Tensor
import torch.nn.functional as F
import torch.nn as nn
from functools import partial
import numpy as np
import logging
from typing import Optional, List
from collections import OrderedDict
from timm.models.registry import register_model
from timm.models.resnet import resnet26d, resnet50d, resnet18d, resnet26, resnet50, resnet101d
from timm.models import (
    create_model,
    safe_model_name,
    resume_checkpoint,
    load_checkpoint,
    convert_splitbn_model,
    model_parameters,
)
from Demo import DemoEnd2EndNet_baseline
import os
from timm.data import (
    create_dataset,
    create_loader,
    resolve_data_config,
    Mixup,
    FastCollateMixup,
    AugMixDataset,
)

In [2]:
os.environ['project_dir'] = "D:\\xm\\DemoEnd2EndNet"
model = create_model(
        "DemoEnd2EndNet_baseline",
        pretrained=False,
        with_lidar=False,
        only_encoder=True,
    )

d:\xm\DemoEnd2EndNet\venv\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [3]:
model.load_state_dict(torch.load(r'D:\xm\DemoEnd2EndNet\net\output\20230819-221257-DemoEnd2EndNet_baseline-224-less_dataset\model_best.pth.tar')['state_dict'])

<All keys matched successfully>

In [4]:
import PIL.Image as Image
from PIL import ImageTransform

In [ ]:
img_path = r"E:\CarlaDataSet\weathers\Town06_long\routes_town06_long_w9_02_27_11_37_19\rgb_front\0764.jpg"

In [ ]:
img = Image.open(img_path).convert('RGB')

In [ ]:
img

In [ ]:
from torchvision import transforms
transform = transforms.Compose([
    # 重置大小
    transforms.Resize(255),
    transforms.CenterCrop(224),
    # 随机旋转图片
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    # 正则化（降低模型复杂度）
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
img_tensor = transform(img)

In [ ]:
x = img_tensor.unsqueeze(0)
x = model.rgb_backbone.model.patch_embed(x)
x = model.rgb_backbone.model.pos_drop(x)

for level in model.rgb_backbone.model.levels:
    x = level(x)

x = model.rgb_backbone.model.norm(x)
x = x.permute(0, 3, 1, 2)
# x = model.rgb_backbone.model.avgpool(x)

In [ ]:
x.shape

In [ ]:
temp = torch.sum(x, dim=1).detach().numpy()

In [ ]:
temp.shape

In [ ]:
temp = np.uint8(255 * (temp[0] - temp.min()) / (temp.max() - temp.min()))

In [ ]:
import cv2
from  matplotlib import pyplot as plt
img = cv2.imread(img_path)
height, width, _ = img.shape  #读取输入图片的尺寸
heatmap = cv2.applyColorMap(cv2.resize(temp, (width, height)), 1)  #CAM resize match input image size
result = np.uint8(heatmap * 0.3 + img * 0.5)

In [ ]:
result.dtype

In [ ]:
from  matplotlib import pyplot as plt

In [ ]:
plt.imshow(result)
plt.show()

In [ ]:
plt.imshow(result)
plt.show()

In [ ]:
plt.imshow(result)
plt.show()

In [ ]:
plt.imshow(result)
plt.show()

In [ ]:
result.max()

In [ ]:
x.shape

In [ ]:
params = model.brake_pred_head[0]

In [ ]:
x.shape

In [ ]:
params.weight.detach().max()

In [5]:
from timm.data import create_carla_dataset, create_carla_loader

In [6]:
dataset_train = create_carla_dataset(
    'carla',
    root='E:\\CarlaDataSet',
    batch_size=1,
    weathers=[1,2,3,4,5,6,7,8,9,10],

    towns=[1, 2, 3, 4, 6, 10],
    multi_view=True,
    use_junction_data=True,
)

In [7]:
loader_train = create_carla_loader(
    dataset_train,
    input_size=224,
    batch_size=1,
    is_training=True
)

In [8]:
iter = enumerate(loader_train)

In [9]:
batch_idx, (input, target) = iter.__next__()

In [10]:
output = model(input)

In [11]:
x = input
front_image = x["rgb"]
left_image = x["rgb_left"]
right_image = x["rgb_right"]
front_center_image = x["rgb_center"]
measurements = x["measurements"]
target_point = x["target_point"]

if model.direct_concat:
    img_size = front_image.shape[-1]
    left_image = torch.nn.functional.interpolate(
        left_image, size=(img_size, img_size)
    )
    right_image = torch.nn.functional.interpolate(
        right_image, size=(img_size, img_size)
    )
    front_center_image = torch.nn.functional.interpolate(
        front_center_image, size=(img_size, img_size)
    )
    front_image = torch.cat(
        [front_image, left_image, right_image, front_center_image], dim=1
    )



features = model.forward_features(
    front_image,
    left_image,
    right_image,
    front_center_image,
    measurements,
)

bs = front_image.shape[0]



tgt = model.position_encoding(
    torch.ones((bs, 1, 20, 20), device=x["rgb"].device)
)
tgt = tgt.flatten(2)
tgt = torch.cat([tgt, model.query_pos_embed.repeat(bs, 1, 1)], 2)
tgt = tgt.permute(2, 0, 1)

features = torch.cat((features, model.waypoint_embed.repeat(1, bs, 1)), dim=0)
memory = model.encoder(features, mask=model.attn_mask)

hs = model.decoder(model.query_embed.repeat(1, bs, 1), memory, query_pos=tgt)[0]

hs = hs.permute(1, 0, 2)  # Batchsize ,  N, C

traffic_feature = hs[:, :400]

velocity = measurements[:, 6:7].unsqueeze(-1)
velocity = velocity.repeat(1, 400, 32)
traffic_feature_with_vel = torch.cat([traffic_feature, velocity], dim=2)
traffic = model.traffic_pred_head(traffic_feature_with_vel)

memory = memory.permute(1, 0, 2)  # Batchsize ,  N, C

is_junction_feature = memory[:, 11]
traffic_light_state_feature = memory[:, 11]
stop_sign_feature = memory[:, 11]
brake_feature = torch.cat((memory, features.permute(1, 0, 2)), dim=1).permute(0, 2, 1)
waypoints_feature = memory[:, 0:10]

waypoints = model.waypoints_generator(waypoints_feature, target_point)


is_junction = model.junction_pred_head(is_junction_feature)


traffic_light_state = model.traffic_light_pred_head(traffic_light_state_feature)

stop_sign = model.stop_sign_head(stop_sign_feature)

# velocity = measurements[:, 6:7].unsqueeze(-1)
# velocity = self.traffic_pred_head(velocity)

brake_feature = model.brake_gap(brake_feature).squeeze(2)
brake = model.brake_pred_head(brake_feature)



In [13]:
torch.softmax(brake, dim=1)

tensor([[9.9999e-01, 5.1604e-06]], grad_fn=<SoftmaxBackward0>)

In [12]:
brake

tensor([[  0.8932, -11.2813]], grad_fn=<AddmmBackward0>)

In [25]:
model.brake_pred_head[4].weight.detach().shape

torch.Size([2, 7])

In [35]:
model.brake_pred_head

Sequential(
  (0): Linear(in_features=512, out_features=64, bias=True)
  (1): Dropout(p=0.5, inplace=False)
  (2): Linear(in_features=64, out_features=7, bias=True)
  (3): ReLU()
  (4): Linear(in_features=7, out_features=2, bias=True)
)

In [28]:
brake * model.brake_pred_head[4].weight.detach().T

tensor([[-0.0393, -1.9285],
        [-0.1473,  1.4721],
        [-0.2925, -3.0395],
        [-0.2664, -1.9549],
        [ 0.0260,  3.8824],
        [-0.3308,  1.8526],
        [-0.2988,  1.2765]], grad_fn=<MulBackward0>)

In [31]:
leaner = model.brake_pred_head[4]

In [34]:
(brake - leaner.bias) * model.brake_pred_head[4].weight.detach().T

tensor([[-0.0426, -1.9565],
        [-0.1599,  1.4934],
        [-0.3175, -3.0835],
        [-0.2891, -1.9832],
        [ 0.0283,  3.9387],
        [-0.3590,  1.8795],
        [-0.3243,  1.2950]], grad_fn=<MulBackward0>)

In [42]:
brake_feature.mean()

tensor(0.4647, grad_fn=<MeanBackward0>)